参考： [[Kaggle]0から本当に機械学習を理解するために学ぶべきこと～一流のデータサイエンティストを例に～ - Qiita](https://qiita.com/zenonnp/items/9cbb2860505a32059d89)

# ライブラリインポート

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# ↑これはJupyter内で図を描写するためのライブラリ

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# データ読み込み

In [ ]:
train_df = pd.read_csv('./input/train.csv')
test_df = pd.read_csv('./input/test.csv')

# データ構成把握

In [ ]:
print(train_df.columns.values)

![img](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F183674%2F7dff3491-0f16-4f59-8d75-48a93eb781ab.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=da8148e79d460bd2540e2931f3330bd1)

In [ ]:
# preview the data
train_df.head()

In [ ]:
# Get data information

train_df.info()

Ageが714

Cabinが204（明らかに少なすぎるので以後は考慮しない）

Name, Sex, Ticket, Embarkedは数値化されていない

In [ ]:
test_df.info()

Age, Fareが欠損

# データの特徴を把握する

In [ ]:
train_df.describe()

* トレーニングデータセットの合計は891で、タイタニック号に搭乗した乗客の実際の数（2,224）の40％です。
  * へー
* Survived は0または1の値を持つカテゴリカル特徴量です。
  * わかる
* トレーニングデータセットの約38％が実際のSurvived率の32％を代表して生き残りました。
  * へー
* ほとんどの乗客（> 75％）は親または子供と一緒に旅行しなかった。
  * どれ？？
* 乗客のほぼ30％が兄弟姉妹や配偶者を乗せていました。
  * どれ？？
* 運賃は、一部の乗客(< 1%)だけ異常に高く、512ドル払っていました。
  * わかる：Fare
* 65歳から80歳までの高齢者はほとんどいませんでした（< 1％）。
  * 1%はわからんがまあわかる

In [ ]:
train_df.describe(include=['O'])

* nameはデータセット全体で一意です（count = unique = 891）
  * わかる
* 65％が男性（top=male、freq= 577 / count= 891）
  * なるほど
* 客室の値はいくつかのトレーニングデータで二重になっています。 あるいは、複数の乗客が客室を共有しました。
  * わかる：Ticket
* Embarkedは3つの値をとる可能性があるが、 ほとんどの乗客がS港をから乗船しました。(top= S)
  * 644/669がSであってるのかな
* Ticket特徴量では、重複した値（ユニーク= 681 ）の割合（22％）が高い。
  * fm?

# 相関を検証する

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

明らかに相関あり

In [ ]:
train_df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

明らかに相関あり

In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

SibSpとParchは特定の数値では無相関
一部の数値を用いた新しい特徴量を作るべき

# データ可視化